In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as  mpatches
import seaborn as sns
import pandas as pd
from labproject.metrics.sliced_wasserstein import sliced_wasserstein_distance
from labproject.metrics.MMD_torch import compute_rbf_mmd,median_heuristic
from labproject.data import get_distribution
from labproject.utils import set_seed
from torch.distributions import MultivariateNormal, Categorical
set_seed(0)

plt.style.use("../../../matplotlibrc")


In [ ]:
"""
Create a Mixture of 2 Gaussians, that we will use as ground truth
"""

class MO2G:
    def __init__(self):
        self.means = torch.tensor(
            [
                [-3.0,1],
                [3, -1],

            ]
        )
        self.covariances = torch.tensor(
            [
                [[1.0, 0], [0, 1.0]],
                [[1.0, 0], [0, 1.0]]

            ]
        )
        self.weights = torch.tensor([0.2,0.8])

        self.gaussians = [
            MultivariateNormal(mean, covariance)
            for mean, covariance in zip(self.means, self.covariances)
        ]

    def sample(self, sample_shape):
        if isinstance(sample_shape, int):
            sample_shape = (sample_shape,)
        categorical = Categorical(self.weights)
        sample_indices = categorical.sample(sample_shape)
        return torch.stack([self.gaussians[i].sample() for i in sample_indices])

    def log_prob(self, input):
        probs = torch.stack([g.log_prob(input).exp() for g in self.gaussians])
        probs = probs.T * self.weights
        return torch.sum(probs, dim=1).log()



In [ ]:
# Plot some samples

MOG = MO2G()
num_samples = 10000
mixture_samples =MOG.sample(num_samples)
plt.scatter(mixture_samples[:, 0], mixture_samples[:, 1], label="Mixture of Gaussians")
covar = torch.cov(mixture_samples.T)


In [ ]:
# Create Gaussians distrubution that can be used as component for our to optimise mixture

class Gauss(nn.Module):
    def __init__(self,dim):
        super(Gauss, self).__init__()
        self.dim = dim
        self.mean = nn.Parameter(torch.randn(dim))
        self.scale_tril = nn.Parameter(torch.eye(dim))
        self.G = torch.distributions.MultivariateNormal(self.mean, scale_tril=self.scale_tril)

    def sample(self,size):
        with torch.no_grad():
            self.G._unbroadcasted_scale_tril.data.copy_(torch.tril(self.G._unbroadcasted_scale_tril))
        return self.G.rsample((size,))

    def cov(self):
        with torch.no_grad():
            self.G._unbroadcasted_scale_tril.data.copy_(torch.tril(self.G._unbroadcasted_scale_tril))
        return self.G.covariance_matrix.detach()

    def log_prob(self, value):
        with torch.no_grad():
            self.G._unbroadcasted_scale_tril.data.copy_(torch.tril(self.G._unbroadcasted_scale_tril))
        return self.G.log_prob(value)

In [ ]:
# Create a mixture of Gaussians that we will fit to the ground truth distribution

class MOGOpt(nn.Module):
    """The model to optimise"""
    def __init__(self,dim):
        super(MOGOpt, self).__init__()
        self.G1 = Gauss(dim)
        self.G2 = Gauss(dim)
        self.weights = nn.Parameter(torch.tensor([0.5, 0.5]), requires_grad=False)
        self.categorical = Categorical(self.weights)
        self.dim = dim

    def sample(self,size):
        with torch.no_grad():
            self.G1.G._unbroadcasted_scale_tril.data.copy_(torch.tril(self.G1.G._unbroadcasted_scale_tril))
            self.G2.G._unbroadcasted_scale_tril.data.copy_(torch.tril(self.G2.G._unbroadcasted_scale_tril))
        samples = torch.zeros(size,self.dim)
        inds= self.categorical.sample((size,))
        s1=self.G1.G.rsample((sum(inds==0),))
        s2=self.G2.G.rsample((sum(inds==1),))
        samples[inds==0]=s1
        samples[inds==1]=s2
        return samples

    def cov(self):
        with torch.no_grad():
            self.G1.G._unbroadcasted_scale_tril.data.copy_(torch.tril(self.G1.G._unbroadcasted_scale_tril))
            self.G2.G._unbroadcasted_scale_tril.data.copy_(torch.tril(self.G2.G._unbroadcasted_scale_tril))
        return [self.G1.covariance_matrix.detach(),self.G2.covariance_matrix.detach()]

    def log_prob(self, input, labels=None):
        with torch.no_grad():
            self.G1.G._unbroadcasted_scale_tril.data.copy_(torch.tril(self.G1.G._unbroadcasted_scale_tril))
            self.G2.G._unbroadcasted_scale_tril.data.copy_(torch.tril(self.G2.G._unbroadcasted_scale_tril))
        probs = torch.stack([self.G1.log_prob(input).exp(),self.G2.log_prob(input).exp()])
        probs = probs.T * self.weights
        return torch.sum(probs, dim=1).log()

    @torch.no_grad
    def e_step(self, input, update_weights=False):
        prob_1 = self.G1.log_prob(input).exp() * self.weights[0]
        prob_2 = self.G2.log_prob(input).exp() * self.weights[1]
        probs = torch.stack([prob_1, prob_2], dim=-1)
        probs /= probs.sum(dim=-1, keepdim=True)
        randfloat = torch.rand(probs.shape[0])
        clusters = randfloat > probs[:, 0]
        if update_weights:

            weights = torch.mean(probs, dim=0)
            self.weights.data.copy_(weights)
        return clusters

In [ ]:
def c2st_target(samples1, samples2, density1, density2):
    r"""Computes optimal C2ST and resulting classification cross-entropy loss
    for optimization
    """
    density_ratios1 = torch.stack([density1.log_prob(samples1), density2.log_prob(samples1)], dim=-1)
    density_ratios2 = torch.stack([density1.log_prob(samples2), density2.log_prob(samples2)], dim=-1)
    probs = torch.cat([density_ratios1, density_ratios2], dim=0)
    labels = torch.cat([torch.zeros(len(samples1)), torch.ones(len(samples2))], dim=0).long()
    loss = -nn.functional.cross_entropy(probs, labels)
    with torch.no_grad():
        c2st = (sum(density_ratios1[:, 0] >= density_ratios1[:, 1]) + sum(density_ratios2[:, 0] < density_ratios2[:, 1])) / (len(samples1) + len(samples2))
    return loss, c2st

In [ ]:
MOG_model_C2ST = MOGOpt(2)
optimizer= torch.optim.Adam(MOG_model_C2ST.parameters(), lr=1e-2)

In [ ]:
C2ST_samples = MOG_model_C2ST.sample(num_samples).detach().numpy()
plt.scatter(C2ST_samples[:, 0], C2ST_samples[:, 1])


In [ ]:
n_iters = 2000
num_samples = 5000

In [ ]:

for epoch in range(n_iters):
    optimizer.zero_grad()

    toy_samples = MOG.sample(num_samples)
    toy_clusters = MOG_model_C2ST.e_step(toy_samples, update_weights=True)

    loss = 0.
    c2st = 0.
    for cluster_idx in range(2):
        toy_mask = toy_clusters == cluster_idx
        toy_cluster_samples = toy_samples[toy_mask]
        toy_n_samples = toy_mask.sum()
        if cluster_idx == 0:
            model = MOG_model_C2ST.G1
        elif cluster_idx == 1:
            model = MOG_model_C2ST.G2
        model_cluster_samples = model.sample(toy_n_samples)
        cent_loss, true_c2st = c2st_target(model_cluster_samples, toy_cluster_samples, model, MOG.gaussians[cluster_idx])
        loss += cent_loss# * (toy_n_samples / toy_clusters.numel())
        c2st += true_c2st# * (toy_n_samples / toy_clusters.numel())

    # c2st_loss = torch.mean(torch.square(preds - 0.5))
    if epoch % 5 == 0:
        print("Iter: {}     loss: {}     c2st: {}".format(epoch, cent_loss.item(), true_c2st.item()))
    loss.backward()
    optimizer.step()

In [ ]:
# MMD
MOG_model_MMD = MOGOpt(dim=2)

In [ ]:

bandwidth = 1
optimizer = torch.optim.Adam(MOG_model_MMD.parameters(), lr=0.01)
MOG_model_MMD.train()

for epoch in range(n_iters):
    optimizer.zero_grad()

    toy_samples = MOG.sample(num_samples)
    toy_clusters = MOG_model_MMD.e_step(toy_samples, update_weights=True)

    loss = 0.
    c2st = 0.
    for cluster_idx in range(2):
        toy_mask = toy_clusters == cluster_idx
        toy_cluster_samples = toy_samples[toy_mask]
        toy_n_samples = toy_mask.sum()
        if cluster_idx == 0:
            model = MOG_model_MMD.G1
        elif cluster_idx == 1:
            model = MOG_model_MMD.G2
        model_cluster_samples = model.sample(toy_n_samples)
        cent_loss= compute_rbf_mmd(model_cluster_samples, toy_cluster_samples, bandwidth)
        loss += cent_loss# * (toy_n_samples / toy_clusters.numel())

    # c2st_loss = torch.mean(torch.square(preds - 0.5))
    if epoch % 1== 0:
        print("Iter: {}     loss: {}     ".format(epoch, cent_loss.item()))
    loss.backward()
    optimizer.step()
MOG_model_MMD.eval()



In [ ]:
# WS

MOG_model_WS = MOGOpt(dim=2)
optimizer = torch.optim.Adam(MOG_model_WS.parameters(), lr=0.01)
MOG_model_WS.train()

for epoch in range(n_iters):
    optimizer.zero_grad()

    toy_samples = MOG.sample(num_samples)
    toy_clusters = MOG_model_WS.e_step(toy_samples, update_weights=True)

    loss = 0.
    c2st = 0.
    for cluster_idx in range(2):
        toy_mask = toy_clusters == cluster_idx
        toy_cluster_samples = toy_samples[toy_mask]
        toy_n_samples = toy_mask.sum()
        if cluster_idx == 0:
            model = MOG_model_WS.G1
        elif cluster_idx == 1:
            model = MOG_model_WS.G2
        model_cluster_samples = model.sample(toy_n_samples)
        cent_loss= sliced_wasserstein_distance(model_cluster_samples,toy_cluster_samples)#, bandwidth=bandwidth)
        loss += cent_loss# * (toy_n_samples / toy_clusters.numel())

    # c2st_loss = torch.mean(torch.square(preds - 0.5))
    if epoch % 1== 0:
        print("Iter: {}     loss: {}     ".format(epoch, cent_loss.item()))
    loss.backward()
    optimizer.step()
MOG_model_WS.eval()



In [ ]:
# Function for plotting elipses according to covariance matrix

#FROM: https://github.com/joferkington/oost_paper_code/blob/master/error_ellipse.py
from matplotlib.patches import Ellipse

def plot_cov_ellipse(cov, pos, nstd=[1,2], ax=None, **kwargs):
    """
    Plots an `nstd` sigma error ellipse based on the specified covariance
    matrix (`cov`). Additional keyword arguments are passed on to the 
    ellipse patch artist.

    Parameters
    ----------
        cov : The 2x2 covariance matrix to base the ellipse on
        pos : The location of the center of the ellipse. Expects a 2-element
            sequence of [x0, y0].
        nstd : The radius of the ellipse in numbers of standard deviations.
            Defaults to 2 standard deviations.
        ax : The axis that the ellipse will be plotted on. Defaults to the 
            current axis.
        Additional keyword arguments are pass on to the ellipse patch.

    Returns
    -------
        A matplotlib ellipse artist
    """
    def eigsorted(cov):
        vals, vecs = np.linalg.eigh(cov)
        order = vals.argsort()[::-1]
        return vals[order], vecs[:,order]

    if ax is None:
        ax = plt.gca()

    vals, vecs = eigsorted(cov)
    theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
    for std in nstd:
    # Width and height are "full" widths, not radius
        width, height = 2 * std * np.sqrt(vals)
        print(pos,width, height,theta)
        ellip = Ellipse(xy=pos, width=width, height=height, angle=theta, **kwargs)

        ax.add_artist(ellip)
    return ellip

In [ ]:
# Make a grey colormap

# FROM: https://stackoverflow.com/questions/18926031/how-to-extract-a-subset-of-a-colormap-as-a-new-colormap-in-matplotlib
import matplotlib.colors as colors


def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

arr = np.linspace(0, 50, 100).reshape((10, 10))
fig, ax = plt.subplots(ncols=2)

cmap = plt.get_cmap('gist_yarg')
new_cmap = truncate_colormap(cmap, 0, 0.75)
ax[0].imshow(arr, interpolation='nearest', cmap=cmap)
ax[1].imshow(arr, interpolation='nearest', cmap=new_cmap)
plt.show()

In [ ]:
num_samples = 10000

# Create a dataframe for the mixture samples
unif_df = pd.DataFrame({'x': mixture_samples[:, 0], 'y': mixture_samples[:, 1]})

# Plot the probability contours
fig,axs = plt.subplots(1,4, figsize = (6.5,3))
sns.kdeplot(ax=axs[0],data=unif_df, x='x', y='y',cmap="Blues", fill=True,levels=12)#, thresh=.1,)#, color="black",alpha=0.75, levels=5)

alpha_bg=1
sns.kdeplot(ax=axs[1],data=unif_df, x='x', y='y', fill=True,cmap=new_cmap,alpha=alpha_bg, levels=12)
sns.kdeplot(ax=axs[2],data=unif_df, x='x', y='y', fill=True,cmap=new_cmap,alpha=alpha_bg, levels=12)
sns.kdeplot(ax=axs[3],data=unif_df, x='x', y='y', fill=True,cmap=new_cmap,alpha=alpha_bg, levels=12)

std_plot = [np.sqrt(.25),np.sqrt(.5),np.sqrt(1),np.sqrt(2)]
std_plot=[.25,.75,1.5,2.5]

plot_cov_ellipse(MOG_model_WS.weights[0].detach().numpy()*MOG_model_WS.G1.cov().detach().numpy(),MOG_model_WS.G1.mean.detach().numpy(),
                 nstd = std_plot ,ax=axs[1],edgecolor='#cc241d', lw=1.5, facecolor='none')
plot_cov_ellipse(MOG_model_MMD.weights[0]*MOG_model_MMD.G1.cov().detach().numpy(),MOG_model_MMD.G1.mean.detach().numpy(),
                 nstd = std_plot ,ax=axs[3],edgecolor='#eebd35', lw=1.5, facecolor='none')
plot_cov_ellipse(MOG_model_C2ST.weights[0]*MOG_model_C2ST.G1.cov().detach().numpy(),MOG_model_C2ST.G1.mean.detach().numpy(),
                 nstd = std_plot ,ax=axs[2],edgecolor='#458588', lw=1.5, facecolor='none')



plot_cov_ellipse(MOG_model_WS.weights[1].detach().numpy()*MOG_model_WS.G2.cov().detach().numpy(),MOG_model_WS.G2.mean.detach().numpy(),
                 nstd = std_plot ,ax=axs[1],edgecolor='#cc241d', lw=1.5, facecolor='none')
plot_cov_ellipse(MOG_model_MMD.weights[1].detach().numpy()*MOG_model_MMD.G2.cov().detach().numpy(),MOG_model_MMD.G2.mean.detach().numpy(),
                 nstd = std_plot ,ax=axs[3],edgecolor='#eebd35', lw=1.5, facecolor='none')
plot_cov_ellipse(MOG_model_C2ST.weights[1].detach().numpy()*MOG_model_C2ST.G2.cov().detach().numpy(),MOG_model_C2ST.G2.mean.detach().numpy(),
                 nstd = std_plot ,ax=axs[2],edgecolor='#458588', lw=1.5, facecolor='none')


for ax in axs:
    ax.spines[['left', 'bottom']].set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_xlim(-8,5)
    ax.set_xlim(-10,10)

    ax.set_ylim(-7,4)
    
    #make square subplots 
    ax.set_box_aspect(1)
axs[0].set_title(r"$p_{true}$")
axs[1].set_title("SW",color ='#cc241d')
axs[3].set_title(r"$MMD_1$",color ='#eebd35')
axs[2].set_title("C2ST",color ="#458588")

fig.tight_layout()
plt.savefig("mode.pdf")